In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import os


In [13]:

class CNNViT(nn.Module):
    def __init__(self, input_shape=(128, 128, 3), num_classes=2):
        super(CNNViT, self).__init__()

        # CNN Feature Extractor (now 5 blocks)
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(64), nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(128), nn.MaxPool2d(2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(256), nn.MaxPool2d(2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(512), nn.MaxPool2d(2)
        )
        self.conv5 = nn.Sequential(  # ✅ New Block
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(512), nn.MaxPool2d(2)
        )

        # After 5 max-pools: 128 → 64 → 32 → 16 → 8 → 4
        self.flatten_dim = 4 * 4  # 16 patches
        self.embedding_dim = 512

        # Positional Encoding
        self.pos_embedding = nn.Parameter(torch.randn(1, self.flatten_dim, self.embedding_dim))

        # Deeper Transformer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embedding_dim,
            nhead=4,
            dim_feedforward=4 * self.embedding_dim,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=6)  # ✅ 6 Layers

        # Classification head
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(self.embedding_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)  # ✅ New CNN block

        B, C, H, W = x.shape
        x = x.view(B, C, H * W).permute(0, 2, 1)  # (B, P, C)

        x = x + self.pos_embedding  # Positional Encoding
        x = self.transformer(x)

        x = x.permute(0, 2, 1)  # (B, C, P)
        x = self.classifier(x)
        return x


In [14]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# ✅ Path to your data directory
data_dir = r"D:\ML\mini project\data"  # Use raw string or forward slashes

# ✅ Transform (similar to ImageDataGenerator)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # optional: normalize like ImageNet
])

# ✅ Load dataset with ImageFolder
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# ✅ Split into train and validation (80% train, 20% val)
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# ✅ Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# ✅ Class names (same as ['fake', 'real'])
print("Classes:", full_dataset.classes)


Classes: ['fake', 'real']


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNViT().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [16]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        train_acc += (preds == labels).sum().item()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_acc /= len(train_dataset)

    # Validation
    model.eval()
    val_loss, val_acc = 0.0, 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            preds = outputs.argmax(dim=1)
            val_acc += (preds == labels).sum().item()
            val_loss += loss.item()

    val_loss /= len(val_loader)
    val_acc /= len(val_dataset)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")


Epoch 1/20 | Train Acc: 0.6587 | Val Acc: 0.6972
Epoch 2/20 | Train Acc: 0.6886 | Val Acc: 0.6972
Epoch 3/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 4/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 5/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 6/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 7/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 8/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 9/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 10/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 11/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 12/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 13/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 14/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 15/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 16/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 17/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 18/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 19/20 | Train Acc: 0.6907 | Val Acc: 0.6972
Epoch 20/20 | Train Acc: 0.6907 | Val Acc: 0.6972


In [18]:
model.eval()
val_loss, val_acc = 0.0, 0.0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        preds = outputs.argmax(dim=1)
        val_acc += (preds == labels).sum().item()
        val_loss += loss.item()

val_acc /= len(val_dataset)
print(f"✅ Final Validation Accuracy: {val_acc * 100:.2f}%")


✅ Final Validation Accuracy: 69.72%


In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
])

img_path = r"D:\ML\mini project\Will_Smith image"  
image = Image.open(img_path).convert("RGB")
image = transform(image).unsqueeze(0) 

# Load model
model = CNNViT()  
model.load_state_dict(torch.load("cnnvit_model.pth"))  
model.eval()
model.to("cuda")

image = image.to("cuda")
with torch.no_grad():
    outputs = model(image)  
    probabilities = F.softmax(outputs, dim=1)  
    predicted_class = torch.argmax(probabilities, dim=1).item()
    prob = probabilities[0][predicted_class].item()


classes = ['fake', 'real']
print(f"Predicted Class: {classes[predicted_class]} with probability: {prob * 100:.2f}%")


Predicted Class: fake with probability: 68.31%


In [19]:
torch.save(model.state_dict(), "cnnvit_model.pth")


In [23]:
from PIL import Image
from torchvision import transforms
import torch

# Define image transform (must match training)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),              # Converts to [0, 1]
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Same norm as training if used
])

# Load image
img_path = r"D:\ML\mini project\Will_Smith image"
image = Image.open(img_path).convert("RGB")
image = transform(image).unsqueeze(0)  # Add batch dimension (1, C, H, W)


In [25]:
image = image.to("cuda")  # Send image to CUDA if model is on CUDA
with torch.no_grad():
    output = model(image)
    predicted_class = torch.argmax(output, dim=1).item()

# Map to class name
classes = ['fake', 'real']
print(f"Predicted Class: {classes[predicted_class]}")

Predicted Class: fake


In [24]:
model = CNNViT()  # Instantiate model
model.load_state_dict(torch.load("cnnvit_model.pth"))  # Load saved weights
model.eval()  # Set model to evaluation mode
model.to("cuda")  # Move to GPU if using CUDA

CNNViT(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()